In [ ]:
!pip install -U langchain-text-splitters
!pip install -U langchain-community langchain-text-splitters langchain
!pip install pypdf
!pip install -U langchain-huggingface
!pip install -qU langchain-chroma
!pip install -qU langchain
!pip install -qU langchain-groq
!pip install sentence-transformers
!pip install -q gradio

In [ ]:
# DAY 1
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
loader = PyPDFLoader("../content/sample_data/assurance.pdf")
pdf = loader.load()

print(f"Document chargé : {len(pdf)} pages trouvées.")

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,      # Taille cible de chaque morceau
    chunk_overlap = 150,    # Chevauchement
    separators = ["\n\n", "\n", " ", ""] # L'ordre de priorité pour couper
)

chunks = text_splitter.split_documents(pdf)

In [ ]:
print(f"Nombre total de chunks pour {len(pdf)} pages : {len(chunks)}")

lengths = [len(c.page_content) for c in chunks]
import statistics
print(f"Taille moyenne : {statistics.mean(lengths)} caractères")

# Voir à quelle page appartient le chunk n°10
print(f"Le chunk 10 vient de la page : {chunks[10].metadata['page']}")

In [ ]:
# DAY 2
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# Création de la base de données physique
vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=hf_embeddings,
    persist_directory="./chroma_db"
)

In [ ]:
question = "Quelles sont les garanties de mon contrat ?"
question_embedding = hf_embeddings.embed_documents([question])

# Demande les 'k' meilleurs résultats
# Chroma fait l'embedding de la question et le calcul de similarité
docs = vectordb.similarity_search(question, k=1)

print(f"Question : {question}")
print("---")
print(f"Réponse: {docs[0].page_content}")

In [ ]:
# DAY 3
from langchain_groq import ChatGroq
import os


llm = ChatGroq(
    model_name="llama-3.1-8b-instant",
    temperature=0.2
)

In [ ]:
from langchain_classic.chains import RetrievalQA

# Transformer la base Chroma en "chercheur" (retriever)
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

# Créer la chaîne RAG
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", # "stuff" veut dire : "donne tout le texte au LLM"
    retriever=retriever
)

In [ ]:
response = qa_chain.invoke("Quelles sont les garanties de mon contrat ?")
print(response["result"])

In [ ]:
# DAY 4

In [ ]:
import gradio as gr
from langchain_classic.memory import ConversationBufferMemory
from langchain_classic.chains import ConversationalRetrievalChain

# 1. Configurer la mémoire
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [ ]:
# 2. Créer la chaîne avec mémoire
qa_chat = ConversationalRetrievalChain.from_llm(
    llm=llm, # LLM Groq
    retriever=retriever, # retriever Chroma
    memory=memory
)

In [ ]:
# 3. Fonction pour l'interface Gradio
def chat_interactif(message, history):
    response = qa_chat.invoke({"question": message})
    return response["answer"]

In [ ]:
# 4. Lancer l'interface
demo = gr.ChatInterface(
    fn=chat_interactif,
    title="📚 Mon Assistant PDF Intelligent",
    description="Posez des questions sur votre document, je m'en souviendrai !",
    examples=["Fais-moi un résumé", "Quels sont les points clés ?"],
    theme="soft"
)

In [ ]:
demo.launch(share=True) # share=True crée un lien public de 72h !